# Bicing project

If you live in Barcelona you probably know that Bicing is a system of public bikes. Users can take a bike from numerous stations and use it for 30 mins without paying. The only cost is the annual fee of around 50 euros. That's basically what I spent on transport in Barcelona every year. Bicing is definitely one of the best things that Barcelona can offer their inhabitants. 

Since I am very frequent user of Bicing I started noticing patterns that would occur in certain locations at certain times. 
* It's hard to find a bike next to my house (La Sagrera) after 9.30 AM on weekdays but at the same time it's difficult to park it in Poblenou where I work. 
* All the stations close to the beach are full on weekends (specially on sunny days!)
* It's easier to find a bike on a rainy and cold day
* There are significantly more bikes in August.

There are many of those. That's why I decided to prove it with data.

I was collecting data from Bicing website between 22-Nov-2015 and 24-Dec-2016. A scraper running on PythonAnywhere would connect to the website, parse the json file and record the data in MySQL database.

There are a couple of weak points of my analysis:
* it ignores refills, when Bicing employees put bikes in certain stations
* PythonAnywhere was not always stable so there are few holes in the database


In [4]:
import pandas as pd
from glob import glob
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline


### PLAN:

* Create empty dataframe
* Read each file
* Manipulate files
* Append to the newly created dataframe
* Dump to HDF5

In [2]:
df = pd.DataFrame(columns=["id", "status", "bikes", "slots", "zip", "timestamp"])

In [ ]:
for file in glob(pathname=r"*.csv"):
    partial_df = pd.read_csv(file)
    partial_df = partial_df.drop_duplicates()
    partial_df["timestamp"] = partial_df["timeDay"].str.cat(partial_df["timeHour"], sep =" ")
    partial_df["timestamp"] = partial_df["timestamp"].str.replace("0 days ", "")
    partial_df["timestamp"] = pd.to_datetime(partial_df["timestamp"], format="%Y-%m-%d %H:%M")
    partial_df = partial_df.drop(["Unnamed: 0", "address", "addressNumber", "timeHour", "timeDay"], axis=1)
    df  = df.append(partial_df)
    
df.to_hdf("bicing_full.h5", key="table")
df.to_csv("bicing_full.csv")

In [6]:
#%timeit df = pd.read_csv("bicing_full.csv")
df = pd.read_hdf("bicing_full.h5", key="table")

HDF5 generated huge savings in terms of space on disk and read time. In my opinion CSV is just not suitable for bigger amount of data.

##### Size on disk 

* csv: 1142 MB
* hdf5: 695 MB (40% less)

##### read time:
* csv: 25.3s
* hdf5: 13.1s (48% less)

In [7]:
df.describe()


,id,status,bikes,slots,zip,timestamp
count,24106065.0,24106065,24106065.0,24106065.0,24106065.0,24106065
unique,465.0,2,43.0,40.0,41.0,51719
top,496.0,'OPN',0.0,0.0,8005.0,2015-11-23 07:40:00
freq,51841.0,23833410,3374549.0,1207513.0,2177322.0,930
first,NaN,NaN,NaN,NaN,NaN,2015-11-22 23:40:00
last,NaN,NaN,NaN,NaN,NaN,2016-12-24 13:30:00
